In [3]:
import torch
import torch_geometric
import deepdish as dd
from tqdm import tqdm
import copy
from os.path import join
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures, AllChem
from rdkit import RDConfig
from rdkit.Chem.rdmolfiles import MolFromMolFile
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
import random
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU, BatchNorm1d, Dropout
import torch_geometric.transforms as T
from torch_geometric.nn import  GCNConv, global_add_pool, global_mean_pool,GATConv,GINConv
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.data import DataLoader
from torch_geometric.utils import remove_self_loops
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset 
from Models.graphLambda import Net

In [4]:
def add_edges_list(root, code):
    '''
    This function creates the graph that represents a molecule. 
    Input: root: root directory of hydrogen-free sdf files of ligands 'str'.
           code: PDB code of the complex containing the ligand 'str'. 
    Output: edges list 'numpy array'
            
    '''
    ligand_filename = code + "_h.sdf"
    m = MolFromMolFile(join(root, code, ligand_filename))
    atoms1 = [b.GetBeginAtomIdx() for b in m.GetBonds()]
    atoms2 = [b.GetEndAtomIdx() for b in m.GetBonds()]    
    # Edge attributes: distance; SINGLE; DOUBLE; TRIPLE; AROMATIC.
    edge_weights= []
    coords = m.GetConformers()[0].GetPositions()  # Get a const reference to the vector of atom positions
    for b in m.GetBonds():
        if str(b.GetBondType()) == "SINGLE":
            edge_weights.append(1)
        elif str(b.GetBondType()) == "DOUBLE":
            edge_weights.append(2)
        elif str(b.GetBondType()) == "TRIPLE":
            edge_weights.append(3)
        else:
            edge_weights.append(4)
    edge_features = np.array(edge_weights) 
    # since the torch-geometric graphs are directed add reverse direction of edges
    return np.array([atoms1 + atoms2, atoms2 + atoms1]), np.concatenate((edge_features, edge_features), 0)

In [6]:
class PDBbindDataset(InMemoryDataset):
    '''
    PDBbindDataset Class: A dataset class that takes root directory, BPS features, and binding affinity values of 
    samples as input. For each complex, the class stores a graph where node features are precomputed BPS
    features, edge features are bond types and target value is the equivalent binding affinity of the given complex.
    Output: A list of graphs.
    '''
    def __init__(self, root, node_features,activity_csv,transform=None, pre_transform=None):
        self.root = root
        self.node_features = node_features
        self.activity_csv = activity_csv
        super(PDBbindDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ["data.pt"]

    def download(self):
        pass

    def process(self):

        self.node_data = dd.io.load(join(self.root, self.node_features))
        # load csv with activity data and simlarity scores
        self.activity = pd.read_csv(join(self.root, self.activity_csv))
        # create lists of edges and edge descriptors 
        self.edge_indexes = {key: add_edges_list(self.root, key)[0] for key in self.activity.PDB }
        self.edge_data = {key: add_edges_list(self.root, key)[1] for key in self.activity.PDB }
        
        # Read data into huge `Data` list.
        data_list = [Data(x = torch.FloatTensor(self.node_data[key]),
                          edge_index = torch.LongTensor(self.edge_indexes[key]),
                          edge_attr = torch.FloatTensor(self.edge_data[key]),
                          y = torch.FloatTensor([self.activity[self.activity.PDB == key].pk.iloc[0]])) for key in self.activity.PDB ]
      
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
@torch.no_grad()
def test_predictions(model, loader):
    model.eval()
    pred = []
    true = []
    for data in loader:
        data = data.to(device)
        pred += model(data).detach().cpu().numpy().tolist()
        true += data.y.detach().cpu().numpy().tolist()
    return pred, true

### Loading the models

In [8]:
model1= Net() 
model2= Net() 
model3= Net() 
model4= Net() 
model1.load_state_dict(torch.load('GNNs_fusion_rand_split.pt'))
model2.load_state_dict(torch.load('GNNs_fusion_TM_split.pt'))
model3.load_state_dict(torch.load('GNNs_fusion_lig2lig_split.pt'))
model4.load_state_dict(torch.load('GNNs_fusion_interaction_split.pt'))

<All keys matched successfully>

## Results on the CSAR sets

In [22]:
path_to_test_directory = "set1" 
path_to_test_complexes_features =  "set1.h5"
path_to_test_complexes_true_affinity = "QSAR_NRC_HiQ_Set.csv"
set1 = PDBbindDataset(path_to_test_directory
                        , path_to_test_complexes_features
                        , path_to_test_complexes_true_affinity)
test_loader = DataLoader(set1, batch_size=1, shuffle=False)

In [ ]:
pred, true = test_predictions(model1, test_loader)
pred2, _ = test_predictions(model2, test_loader)
pred3, _ = test_predictions(model3, test_loader)
pred4, _ = test_predictions(model4, test_loader)

In [21]:
print("Results on CSAR_set1")
print("RMSE Random split: ",np.sqrt(mean_squared_error(pred,true)))
print("RMSE PPS: ",np.sqrt(mean_squared_error(pred2,true)))
print("RMSE LLS: ",np.sqrt(mean_squared_error(pred3,true)))
print("RMSE CCS: ",np.sqrt(mean_squared_error(pred4,true)))

Results on CSAR_set1
RMSE Random split:  1.22
RMSE PPS:  1.38
RMSE LLS:  1.26
RMSE CCS:  1.2700000000000002


In [ ]:
path_to_test_directory = "set2"  
path_to_test_complexes_features =  "set2.h5"
path_to_test_complexes_true_affinity = "QSAR_NRC_HiQ_Set.csv"
set2 = PDBbindDataset(path_to_test_directory
                        , path_to_test_complexes_features
                        , path_to_test_complexes_true_affinity)
test_loader = DataLoader(set2, batch_size=1, shuffle=False)

In [ ]:
pred, true = test_predictions(model1, test_loader)
pred2, _ = test_predictions(model2, test_loader)
pred3, _ = test_predictions(model3, test_loader)
pred4, _ = test_predictions(model4, test_loader)

In [19]:
print("Results on CSAR_set2")
print("RMSE Random split: ",np.sqrt(mean_squared_error(pred,true)))
print("RMSE PPS: ",np.sqrt(mean_squared_error(pred2,true)))
print("RMSE LLS: ",np.sqrt(mean_squared_error(pred3,true)))
print("RMSE CCS: ",np.sqrt(mean_squared_error(pred4,true)))

Results on CSAR_set2
RMSE Random split:  1.0099999999999998
RMSE PPS:  1.11
RMSE LLS:  1.18
RMSE CCS:  1.09
